In [18]:
import pandas as pd
import gspread
import requests

token = 'AF5y90PW7vmJALond4SEGVBFwmt3Jhj7Ba7oclAeXL4'

In [19]:
sa = gspread.service_account(filename='geprek-wani-abcd00886732.json')

In [20]:
sheet = sa.open('Wani Spreadsheet')

In [21]:
database = sheet.worksheet('Database Harga')

In [22]:
df_harga = pd.DataFrame(database.get_all_records())

In [23]:
worksheet = sheet.worksheet('Pesanan')

In [24]:
df = pd.DataFrame(worksheet.get_all_records())

In [25]:
df.rename(columns={'Request cabe GEPREK atau cabe TERIYAKI': 'Cabe', 
                   'Sambal Tambahan (Rp 2.000,-)': 'Sambal',
                   'Camilan (Rp 10.000,-)': 'Camilan',
                   }, inplace=True)

In [26]:
df.head()

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 0182226503 a/n AVERILL SALADIN ATMA SETIAWAN","Tambah Nasi Putih (Rp 5.000,-)",Bayar?,
0,09/06/2023 09:05:58,Tot,"Nasi Ayam Teriyaki ~ Rp 15.000,-",0,,"Es Teh Jumbo ~ Rp 5.000,-",,https://drive.google.com/open?id=1GkIdb-C4jofN...,,,
1,09/06/2023 09:06:53,Max,"Nasi Ayam Geprek ~ Rp 12.000,-",,,,,https://drive.google.com/open?id=1qQ1vGUW7OKyJ...,,,
2,09/06/2023 09:07:19,JoWi,"Nasi Penyet Tahu Telur Tempe ~ Rp 13.000,-",,,,,https://drive.google.com/open?id=10Ur_-zDleC5f...,,,
3,09/06/2023 09:07:25,Abel,"Nasi Ayam Bakar ~ Rp 15.000,-",,,,,https://drive.google.com/open?id=1Fj5OqjrYEHzg...,,,
4,09/06/2023 09:08:29,Rey,"Indomie Telur Geprek ~ Rp 12.000,-",0,,"Es Teh Jumbo ~ Rp 5.000,-",,https://drive.google.com/open?id=1BBG7MkfDQGea...,,,


In [27]:
# create a list containing 'Makanan Utama' and 'Nama'
list = ['Makanan Utama', 'Cabe', 'Nama']

# replace the empty string with 1 if Makanan Utama is not empty
df['Cabe'] = df['Cabe'].apply(lambda x: 1 if x == '' else x)

# remove everything after ' ~' in Makanan Utama
df['Makanan Utama'] = df['Makanan Utama'].apply(lambda x: x.split(' ~')[0])
df['Minuman'] = df['Minuman'].apply(lambda x: x.split(' ~')[0])

df[list]

,Makanan Utama,Cabe,Nama
0,Nasi Ayam Teriyaki,0,Tot
1,Nasi Ayam Geprek,1,Max
2,Nasi Penyet Tahu Telur Tempe,1,JoWi
3,Nasi Ayam Bakar,1,Abel
4,Indomie Telur Geprek,0,Rey
5,Nasi Ayam Geprek,1,Angeline
6,Nasi Ayam Crisbar,1,Oedin
7,Nasi Penyet Tahu Telur Tempe,1,MC
8,Kwetiau Goreng,1,Timothy
9,Nasi Penyet Tahu Telur Tempe,1,MW


In [28]:
# Custom geprek
df_geprek = df[df['Makanan Utama'].str.contains('Geprek', case=False) & ~df['Makanan Utama'].str.contains('Indomie') & ~df['Makanan Utama'].str.contains('Udang')]
df_geprek

,Timestamp,Nama,Makanan Utama,Cabe,Camilan,Minuman,Sambal,"Bukti Pembayaran, transfer ke BCA: 0182226503 a/n AVERILL SALADIN ATMA SETIAWAN","Tambah Nasi Putih (Rp 5.000,-)",Bayar?,
1,09/06/2023 09:06:53,Max,Nasi Ayam Geprek,1,,,,https://drive.google.com/open?id=1qQ1vGUW7OKyJ...,,,
5,09/06/2023 09:09:00,Angeline,Nasi Ayam Geprek,1,,,,https://drive.google.com/open?id=1-UWhfDw6LW6f...,,,
17,,Kenny,Nasi Ayam Geprek Keju,0,,,,,,,


In [29]:
# Dataframe not-geprek but include 'Indomie' and 'Udang' and doen't include 'Teriyaki'
df_not_geprek = df[~df['Makanan Utama'].str.contains('geprek', case=False)]
# drop rows that contains 'Teriyaki'
df_not_geprek = df_not_geprek[~df_not_geprek['Makanan Utama'].str.contains('Teriyaki')]

# drop rows with empty Makanan Utama
df_not_geprek = df_not_geprek[df_not_geprek['Makanan Utama'] != '']

# Dataframe geprek tanpa nasi
df_geprek_tanpa_nasi = df_geprek[df_geprek['Makanan Utama'].str.contains('tanpa Nasi')]

# Dataframe geprek dengan nasi
df_geprek_dengan_nasi = df_geprek[~df_geprek['Makanan Utama'].str.contains('tanpa Nasi') & ~df_geprek['Makanan Utama'].str.contains('Keju')]

# Dataframe indomie geprek
df_indomie_geprek = df[df['Makanan Utama'].str.contains('Indomie') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe udang geprek
df_udang_geprek = df[df['Makanan Utama'].str.contains('Udang') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe geprek keju
df_geprek_keju = df[df['Makanan Utama'].str.contains('Keju') & df['Makanan Utama'].str.contains('Geprek')]

# Dataframe camilan
df_camilan = df[['Camilan', 'Nama']]
df_camilan = df_camilan[df_camilan['Camilan'] != '']

# Dataframe minuman
df_minuman = df[['Minuman', 'Nama']]
df_minuman = df_minuman[df_minuman['Minuman'] != '']

# Dataframe sambal
df_sambal = df[['Sambal', 'Nama']]
df_sambal = df_sambal[df_sambal['Sambal'] != '']

# Dataframe teriyaki
df_teriyaki = df[df['Makanan Utama'].str.contains('Teriyaki')]

In [30]:
combined_prints = []
combined_prints.append(f"Pesanan Depot Wani: {pd.Timestamp.now()}")
combined_prints.append("=====================================")
combined_prints.extend([f"{sum(df_geprek_dengan_nasi['Cabe'] == i)} Nasi Ayam Geprek Cabe {i} ({', '.join(df_geprek_dengan_nasi[df_geprek_dengan_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_dengan_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_keju['Cabe'] == i)} Nasi Ayam Geprek Keju Cabe {i} ({', '.join(df_geprek_keju[df_geprek_keju['Cabe'] == i]['Nama'])})" for i in df_geprek_keju['Cabe'].unique()])
combined_prints.extend([f"{sum(df_teriyaki['Cabe'] == i)} Nasi Ayam Teriyaki Cabe {i} ({', '.join(df_teriyaki[df_teriyaki['Cabe'] == i]['Nama'])})" for i in df_teriyaki['Cabe'].unique()])
combined_prints.extend([f"{sum(df_geprek_tanpa_nasi['Cabe'] == i)} Ayam Geprek tanpa nasi Cabe {i} ({', '.join(df_geprek_tanpa_nasi[df_geprek_tanpa_nasi['Cabe'] == i]['Nama'])})" for i in df_geprek_tanpa_nasi['Cabe'].unique()])
combined_prints.extend([f"{sum(df_not_geprek['Makanan Utama'] == i)} {i} ({', '.join(df_not_geprek[df_not_geprek['Makanan Utama'] == i]['Nama'])})" for i in df_not_geprek['Makanan Utama'].unique()])
combined_prints.extend([f"{sum(df_indomie_geprek['Cabe'] == i)} {df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_indomie_geprek[df_indomie_geprek['Cabe'] == i]['Nama'])})" for i in df_indomie_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_udang_geprek['Cabe'] == i)} {df_udang_geprek[df_udang_geprek['Cabe'] == i]['Makanan Utama'].iloc[0]} Cabe {i} ({', '.join(df_udang_geprek[df_udang_geprek['Cabe'] == i]['Nama'])})" for i in df_udang_geprek['Cabe'].unique()])
combined_prints.extend([f"{sum(df_camilan['Camilan'] == i)} {i} ({', '.join(df_camilan[df_camilan['Camilan'] == i]['Nama'])})" for i in df_camilan['Camilan'].unique()])
combined_prints.extend([f"{sum(df_minuman['Minuman'] == i)} {i} ({', '.join(df_minuman[df_minuman['Minuman'] == i]['Nama'])})" for i in df_minuman['Minuman'].unique()])
combined_prints.extend([f"{sum(df_sambal['Sambal'] == i)} {i} ({', '.join(df_sambal[df_sambal['Sambal'] == i]['Nama'])})" for i in df_sambal['Sambal'].unique()])

print('\n'.join(combined_prints))

Pesanan Depot Wani: 2023-06-09 10:45:59.324033
2 Nasi Ayam Geprek Cabe 1 (Max, Angeline)
1 Nasi Ayam Geprek Keju Cabe 0 (Kenny)
1 Nasi Ayam Teriyaki Cabe 0 (Tot)
3 Nasi Penyet Tahu Telur Tempe (JoWi, MC, MW)
3 Nasi Ayam Bakar (Abel, zefa, Jalu)
1 Nasi Ayam Crisbar (Oedin)
1 Kwetiau Goreng (Timothy)
1 Nasi Goreng Telor (zefa)
1 Nasi Fuyunghai (kevin)
1 Nasi Ayam Goreng Lengkuas (Irfan)
1 Mie Goreng Biasa (Alex)
1 Nasi Goreng Kulit Jumbo (Verill)
1 Indomie Telur Geprek Cabe 0 (Rey)
2 Es Teh Jumbo (Tot, Rey)


In [31]:
# remove all text after '(' in combined_prints
# combined_prints = [i.split('(')[0] for i in combined_prints]

# print('\n'.join(combined_prints))

In [32]:
# payload = {'message' : '\n'.join(combined_prints)}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload)

In [33]:
# duar
# payload = {'message' : 'Duar'}
# files = {'imageFile': open('duar.jpg', 'rb')}
# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)

In [34]:
# ambatukam
# payload = {'message' : 'Ambatukam @Jason'}
# files = {'imageFile': open('ambatukam.jpg', 'rb')}

# r = requests.post('https://notify-api.line.me/api/notify'
#                 , headers={'Authorization' : 'Bearer {}'.format(token)}
#                 , params = payload, files=files)